In [1]:
# import libraries
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns 
import html
from bs4 import BeautifulSoup
import requests
import os
from selenium import webdriver
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import warnings; warnings.simplefilter('ignore')
from VE_scraper_functions import *
from chromedriver_py import binary_path # this will get you the path variable
from selenium.webdriver.common.by import By
import time
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Daniel.Lang\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [2]:
path = "park_scrape_content_dataset.xlsx"
park_scrape_dataset2 = pd.read_excel(path)


In [3]:
stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", 
             "you", "your", "yours", "yourself", "yourselves", "he", "him", 
             "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
             "they", "them", "their", "theirs", "themselves", "what", "which", "who",
             "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
             "were", "be", "been", "being", "have", "has", "had", "having", "do", 
             "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", 
             "because", "as", "until", "while", "of", "at", "by", "for", "with", "about",
             "against", "between", "into", "through", "during", "before", "after", "above", 
             "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", 
             "again", "further", "then", "once", "here", "there", "when", "where", "why", 
             "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", 
             "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", 
             "s", "t", "can", "will", "just", "don", "should", "now",
            "href","=","/",">","<","]","[","span","'\n'",'class',"jstcache",
            "onclick","null","jscontent"," <br/>","</span>",",",";","(",")","{","}",":","''",
            "&","'","var","+=",".","#","-","=","+","``","0","’","data.operatingHours","outputVarOperatingHours",
            ".exceptions","--","1","-1","?","class=","==","div","/div","$","li","e","!","k","/span","jQuery",
            "tabindex",'j','l']

In [14]:
## Model to calculate VE fields ##

#create new sheet so with our variables for each park
park_sheet = pd.DataFrame(columns = ['park', 'Driving_Directions','Public_transportation_information',
                                     'Bike_Pedestrian_Information','Congestion_information','Accessibility',
                                           'Alternative_Fueling_Stations', 'website page count'])
z=0
tic = time.process_time() #function to let us track processing time


for x in park_scrape_dataset2['park'].unique():
    z+=1
    if z % 25 == 0: 
        #function to let us track processing time
        z5 = 424-z
        toc = time.process_time()
        time_diff = toc-tic
        print("Current Park: ", x, ": ", z, " checks done; ", z5, " remaining; Processing Time: ",time_diff)
        tic=toc
        
    this_park = park_scrape_dataset2[(park_scrape_dataset2['park']==x)] #filter our webscraping dataset for our park's website code
    park_final = Traveler_Info_Finder(this_park) #run function
    print(park_final['Accessibility_information'])
    park_sheet = park_sheet.append({'park': park_final._get_value(0,'park'),
                        'website page count': park_final._get_value(0,'website page'),
                        'Directions_word_count': park_final._get_value(0,'Directions_count'),
                        'Directions_page_count': park_final._get_value(0,'Directions_page_count'),
                       'Driving_Directions': park_final._get_value(0,'MajorDirections_count'),
                       'Public_transportation_information': park_final._get_value(0,'Public_transportation_information'),
                       'Alternative_Fueling_Stations': park_final._get_value(0,'Alternative_Fueling_Stations'), 
                       'Bike_Pedestrian_Information': park_final._get_value(0,'Bike_Pedestrian_Information'),
                       'Congestion_information': park_final._get_value(0,'Congestion_information'),
                        'Travel_Distance_Information': park_final._get_value(0,'Travel_dist_information'),
                        'Travel_other_dist_information': park_final._get_value(0,'Travel_other_dist_information'),
                        'Accessibility': park_final._get_value(0,'Accessibility_information'),
                        'Parking_raw_information': park_final._get_value(0,'Parking_information'),
                        'Parking_experience_information': park_final._get_value(0,'Parking_experience_information'),
                        'Parking_max_on_one_site': park_final._get_value(0,'Parking_max_on_one_site')
                       },
                      ignore_index=True)
    park_sheet.loc[park_sheet.Driving_Directions > 0, 'Driving_Directions'] = 1
    park_sheet.loc[park_sheet.Alternative_Fueling_Stations > 0, 'Alternative_Fueling_Stations'] = 1
    park_sheet.loc[park_sheet.Public_transportation_information > 0, 'Public_transportation_information'] = 1
    park_sheet.loc[park_sheet.Bike_Pedestrian_Information > 0, 'Bike_Pedestrian_Information'] = 1
    park_sheet.loc[park_sheet.Congestion_information > 0, 'Congestion_information'] = 1
    park_sheet.loc[park_sheet.Accessibility > 0, 'Accessibility'] = 1
 #   park_sheet.loc[park_sheet.Parking_information > 0, 'Parking_information'] = 1
    park_sheet['Travel_Distance_Final']=np.where(
        np.logical_or(park_sheet['Travel_Distance_Information']>9, 
                     park_sheet['Travel_other_dist_information']>0),1,0)
    park_sheet['Parking_Experience_information']=np.where((
        park_sheet['Parking_raw_information']/park_sheet['website page count'])>0.25,1,0)
    park_sheet['Transportation_experience_information']=np.where((
        park_sheet['Directions_page_count']/park_sheet['website page count'])>0.65,1,0)
    park_sheet['Parking_information']=np.where(np.logical_or(
        park_sheet['Parking_Experience_information']==1,
        park_sheet['Parking_max_on_one_site']>2),1,0)


park_sheet= park_sheet.drop(columns=['website page count', 'Directions_word_count',
                        'Directions_page_count','Parking_raw_information','Parking_experience_information',
                        'Parking_max_on_one_site','Travel_Distance_Information','Travel_other_dist_information'])
    
#create csv
park_sheet.to_csv("final_park.csv") #save final csv
os.system("start EXCEL.EXE final_park.csv") #open csv file

0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype: int64
0    0
Name: Accessibility_information, dtype:

KeyboardInterrupt: 

In [5]:
park_final

,park,MajorDirections_count,Directions_count,Directions_page_count,Public_transportation_information,Alternative_Fueling_Stations,Bike_Pedestrian_Information,Congestion_information,Travel_dist_information,Travel_other_dist_information,Accessibility_information,Parking_information,Parking_experience_information,Parking_max_on_one_site,website page
0,ZION,22,0,22,16,0,3,1,10,0,0,17,0,0,22


In [11]:
park_sheet.to_csv("final_park.csv") #save final csv
os.system("start EXCEL.EXE final_park.csv") #open csv file

0

In [20]:
this_park = park_scrape_dataset2[(park_scrape_dataset2['park']=='ZION')]

Accessibility_Words = [
        "wheelchair", "accessibility", "disability", "impaired", "disabilities", "handicap",
        "accessible","Wheelchair"
    ]

Directions_Words = ["Entrance","Center","street","Visitor"
                    "Street","parking","directions","Route","Road",
                    "Interstate","Exit",
                    "mile","km","ferry","access", "Street","Blvd", "Hwy"
                   ]

Parking_Words = [
    "parking", "Parking", "pullout"
]
Direction_count = []
Congestion_count = []
Travel_dist_count = []
Travel_dist_other_count=[]
Accessibility_count=[]
Parking_count=[]
Parking_plan_count=[]

Accessibility_information_count = []

for x in this_park['content']:
    z=0
    z2=0
    z3=0


    major = 0
    congestion = 0
    pubtrans=0
    bikeped=0
    try:
        tokenized_word=word_tokenize(x)
        filtered_sent=[]
        stemmed_words=[]
        direction_words_temp = []
        for w in tokenized_word:
            if w not in stopwords:
                filtered_sent.append(w)
        for w in filtered_sent:
            if w in Directions_Words:
                z += 1
            if w in Parking_Words:
                z2 +=1
            if w in Accessibility_Words:
                z3 += 1
        Direction_count.append(z)
        Parking_plan_count.append(z2)
        Accessibility_count.append(z3)
        Accessibility_info = np.where(np.logical_and(z2>2,z3>2),1,0)
        Accessibility_information_count.append(Accessibility_info)
    except:
        Direction_count.append(0)
        Parking_plan_count.append(0)
        Accessibility_count.append(0)

In [21]:
np.logical_and(z2>2,z3>2)

False

In [23]:
Accessibility_information_count

[array(0),
 array(0),
 array(0),
 array(0),
 array(0),
 array(0),
 array(0),
 array(0),
 array(0),
 array(0),
 array(0),
 array(0),
 array(0),
 array(0),
 array(0),
 array(0),
 array(0),
 array(1),
 array(0),
 array(0),
 array(0),
 array(0)]

In [22]:
sum(Accessibility_information_count)

1

In [17]:
z3

1

In [18]:
np.where(
        np.logical_or(Accessibility_count>2,
                     Parking_plan_count>2),1,0)

TypeError: '>' not supported between instances of 'list' and 'int'

In [ ]:
Accessibility_count